In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!wget https://github.com/NVlabs/earth2grid/archive/refs/tags/v2024.8.1.tar.gz

--2025-05-08 18:37:26--  https://github.com/NVlabs/earth2grid/archive/refs/tags/v2024.8.1.tar.gz
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/NVlabs/earth2grid/tar.gz/refs/tags/v2024.8.1 [following]
--2025-05-08 18:37:26--  https://codeload.github.com/NVlabs/earth2grid/tar.gz/refs/tags/v2024.8.1
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘v2024.8.1.tar.gz’

v2024.8.1.tar.gz        [ <=>                ] 152.13K  --.-KB/s    in 0.1s    

2025-05-08 18:37:27 (1.03 MB/s) - ‘v2024.8.1.tar.gz’ saved [155784]



In [3]:
!tar -xvf v2024.8.1.tar.gz

earth2grid-2024.8.1/
earth2grid-2024.8.1/.bumpversion.cfg
earth2grid-2024.8.1/.editorconfig
earth2grid-2024.8.1/.github/
earth2grid-2024.8.1/.github/ISSUE_TEMPLATE.md
earth2grid-2024.8.1/.github/workflows/
earth2grid-2024.8.1/.github/workflows/ci.yml
earth2grid-2024.8.1/.gitignore
earth2grid-2024.8.1/.gitlab-ci.yml
earth2grid-2024.8.1/.pre-commit-config.yaml
earth2grid-2024.8.1/CHANGELOG.md
earth2grid-2024.8.1/CONTRIBUTING.md
earth2grid-2024.8.1/LICENSE.txt
earth2grid-2024.8.1/README.md
earth2grid-2024.8.1/docs/
earth2grid-2024.8.1/docs/.gitignore
earth2grid-2024.8.1/docs/Makefile
earth2grid-2024.8.1/docs/api.rst
earth2grid-2024.8.1/docs/changelog.md
earth2grid-2024.8.1/docs/conf.py
earth2grid-2024.8.1/docs/contributing.md
earth2grid-2024.8.1/docs/image_scraper.py
earth2grid-2024.8.1/docs/img/
earth2grid-2024.8.1/docs/img/image.jpg
earth2grid-2024.8.1/docs/index.rst
earth2grid-2024.8.1/docs/installation.md
earth2grid-2024.8.1/docs/make.bat
earth2grid-2024.8.1/docs/push_docs.sh
earth2gr

In [ ]:
!pip install --no-build-isolation /content/earth2grid-2024.8.1

Processing ./earth2grid-2024.8.1
  Preparing metadata (pyproject.toml) ... done


# Healpix to lat-lon

In [ ]:
import earth2grid
import torch
level = 6
hpx = earth2grid.healpix.Grid(level=level, pixel_order=earth2grid.healpix.XY())
src = earth2grid.latlon.equiangular_lat_lon_grid(32, 64)
z_torch = torch.cos(torch.deg2rad(torch.tensor(src.lat)))
z_torch = z_torch.broadcast_to(src.shape)
regrid = earth2grid.get_regridder(src, hpx)
z_hpx = regrid(z_torch)
nside = 2**level
reshaped = z_hpx.reshape(12, nside, nside)
lat_r = hpx.lat.reshape(12, nside, nside)
lon_r = hpx.lon.reshape(12, nside, nside)

In [ ]:
import numpy as np
import torch

import earth2grid

device = "cpu"


# the source grid (from 90N to 90S and 0E to 360E)
ll = earth2grid.latlon.equiangular_lat_lon_grid(721, 1440)

# a 2d grid of target lat lons
target_lat = np.linspace(30, 50, 32)
target_lon = np.linspace(100, 120, 64)
target_lat, target_lon = np.meshgrid(target_lat, target_lon)

# Some source data on the original grid
data = torch.ones([721, 1440]).to(device)

# Create a bilinear regridding object earth2grid
regrid = ll.get_bilinear_regridder_to(target_lat, target_lon)

# need to move the weights to same device and dtype as data
regrid.to(data)

# perform the regridding
out = regrid(data)
assert out.shape == target_lat.shape  # noqa
print("data shape", out.shape)

# Lat-lon to Healpix

In [ ]:
import torch
import earth2grid

def latlon_to_healpix(data: torch.Tensor, level: int = 6, device: str = "cpu"):
    """
    Regrid 2D lat-lon data to HEALPix format using earth2grid.

    Args:
        data (torch.Tensor): 2D tensor of shape [nlat, nlon].
        level (int): HEALPix resolution level (default: 6 → nside=64).
        device (str): Device to perform computation on ("cpu" or "cuda").

    Returns:
        data_hpx (torch.Tensor): Flat HEALPix data of shape [12 * nside * nside].
        data_faces (torch.Tensor): Reshaped HEALPix data of shape [12, nside, nside].
    """
    # Make sure data is on the correct device
    data = data.to(device)

    # Get source grid matching data shape
    nlat, nlon = data.shape
    src = earth2grid.latlon.equiangular_lat_lon_grid(nlat, nlon)

    # Create HEALPix target grid
    hpx = earth2grid.healpix.Grid(level=level, pixel_order=earth2grid.healpix.XY())
    nside = 2 ** level

    # Create and prepare regridder
    regrid = earth2grid.get_regridder(src, hpx)
    regrid.to(data)

    # Perform regridding
    data_hpx = regrid(data)
    data_faces = data_hpx.reshape(12, nside, nside)

    return data_hpx, data_faces

# Example data: 721 x 1440 grid
data = torch.ones([721, 1440])

# Regrid to HEALPix level 6
data_hpx, data_faces = latlon_to_healpix(data, level=6)

print("Flat HEALPix shape:", data_hpx.shape)       # torch.Size([49152])
print("Face-wise shape:", data_faces.shape)        # torch.Size([12, 64, 64])

In [ ]:
!wget https://raw.githubusercontent.com/manmeet3591/download_mars_reanalysis/refs/heads/main/max_vals_mars.nc
!wget https://raw.githubusercontent.com/manmeet3591/download_mars_reanalysis/refs/heads/main/min_vals_mars.nc

In [ ]:
import xarray as xr
ds_min = xr.open_dataset('min_vals_mars.nc')
ds_max = xr.open_dataset('max_vals_mars.nc')

In [ ]:
!ls /content/drive/MyDrive/mars_reanalysis/

In [ ]:
import xarray as xr
import glob

# Path to your directory containing the NetCDF files
path = '/content/drive/MyDrive/mars_reanalysis/'

# Get a list of all NetCDF files in the directory
file_list = sorted(glob.glob(path + 'emars_*.nc'))

# # Optionally: create a list of datasets
# datasets = []

for file in file_list[:1]:
    ds = xr.open_dataset(file)
    ds_normalized = (ds - ds_min) / (ds_max - ds_min)

In [ ]:
ds_normalized

In [ ]:
ds_normalized.T

In [ ]:
import torch
import xarray as xr
from tqdm import tqdm
import earth2grid

def latlon_to_healpix(data: torch.Tensor, level: int = 6, device: str = "cpu"):
    data = data.to(device)
    nlat, nlon = data.shape
    src = earth2grid.latlon.equiangular_lat_lon_grid(nlat, nlon)
    hpx = earth2grid.healpix.Grid(level=level, pixel_order=earth2grid.healpix.XY())
    nside = 2 ** level
    regrid = earth2grid.get_regridder(src, hpx)
    regrid.to(data)
    data_faces = regrid(data).reshape(12, nside, nside)
    return data_faces

def convert_to_healpix_array(ds: xr.DataArray, level: int = 6, device: str = "cpu") -> torch.Tensor:
    """
    Convert xarray.DataArray [time, pfull, lat, lon]
    into torch.Tensor [time, pfull, 12, nside, nside].
    """
    time_len = ds.sizes["time"]
    pfull_len = ds.sizes["pfull"]
    nside = 2 ** level

    output = torch.empty((time_len, pfull_len, 12, nside, nside), dtype=torch.float32)

    for t in tqdm(range(time_len), desc="Time"):
        for p in range(pfull_len):
            latlon = torch.tensor(ds.isel(time=t, pfull=p).values, dtype=torch.float32)
            hp = latlon_to_healpix(latlon, level=level, device=device)
            output[t, p] = hp

    return output

In [ ]:
healpix_tensor = convert_to_healpix_array(ds_normalized.T, level=6, device="cpu")
print(healpix_tensor.shape)  # torch.Size([time, pfull, 12, 64, 64])

In [ ]:
healpix_tensor[:1].shape

In [ ]:
# Step 1: Convert each to HEALPix
healpix_T = convert_to_healpix_array(ds_normalized.T, level=6, device="cpu")
healpix_U = convert_to_healpix_array(ds_normalized.U, level=6, device="cpu")
healpix_V = convert_to_healpix_array(ds_normalized.V, level=6, device="cpu")

# Step 2: Concatenate along the pfull dimension (dim=1)
healpix_combined = torch.cat([healpix_T, healpix_U, healpix_V], dim=1)

print("Final shape:", healpix_combined.shape)
# Expected shape: [time, 3 * pfull, 12, 64, 64]